# Туториал по PyTorch: от установки до готовой нейронной сети

https://neurohive.io/ru/tutorial/glubokoe-obuchenie-s-pytorch/

## Тензоры

In [279]:
import torch
from torch.autograd import Variable

In [280]:
torch.Tensor(2, 3)

tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000]])

In [281]:
torch.rand(2, 3)

tensor([[0.3492, 0.7663, 0.6051],
        [0.7126, 0.7209, 0.0406]])

In [282]:
x = torch.ones(2,3)
y = torch.ones(2,3) * 2
x + y

tensor([[3., 3., 3.],
        [3., 3., 3.]])

In [283]:
y[:,1] = y[:,1] + 1
y

tensor([[2., 3., 2.],
        [2., 3., 2.]])

## Автоматическое дифференцирование в PyTorch

In [284]:
x = Variable(torch.ones(2, 2) * 2, requires_grad=True) 
# Если последний параметр будет равен False, то переменная не может использоваться для обучения
x

tensor([[2., 2.],
        [2., 2.]], requires_grad=True)

In [285]:
z = 2 * (x * x) + 5 * x
z

tensor([[18., 18.],
        [18., 18.]], grad_fn=<ThAddBackward>)

In [286]:
z.backward(torch.ones(2, 2))
x.grad

tensor([[13., 13.],
        [13., 13.]])

## Класс для построения нейронной сети

In [287]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1) 

# UserWarning: Implicit dimension choice for log_softmax has been deprecated. 
# Change the call to include dim=X as an argument

In [288]:
net = Net()
# Нейрон смещения или bias нейрон — это третий вид нейронов, используемый в большинстве нейросетей. 
# Особенность этого типа нейронов заключается в том, что его вход и выход всегда равняются 1 
# и они никогда не имеют входных синапсов.
net

Net(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)

## Тренировка сети

In [289]:
def train(optimizer, batch_size=200, epochs=10, log_interval=10):
    # запускаем главный тренировочный цикл
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = Variable(data), Variable(target)

            # изменим размер с (batch_size, 1, 28, 28) на (batch_size, 28*28)
            data = data.view(-1, 28*28)
            optimizer.zero_grad()
            net_out = net(data)
            loss = criterion(net_out, target)
            loss.backward()
            optimizer.step()
            
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                   epoch, batch_idx * len(data), len(train_loader.dataset),
                          100. * batch_idx / len(train_loader), loss.data.item())) # loss.data[0]
                      

## Train and Test data

In [290]:
from torchvision import datasets, transforms

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)

## Check network

In [291]:
def main():
    test_loss = 0
    correct = 0
    
    # instead of Variable(data, volatile=True)
    # https://discuss.pytorch.org/t/volatile-now-has-no-effect-use-with-torch-no-grad-instead/26656
    with torch.no_grad(): 
        
        # evaluate
        for data, target in test_loader:
            data = data.view(-1, 28 * 28)
            net_out = net(data)

            # Суммируем потери со всех партий
            test_loss += criterion(net_out, target).data.item() # data[0]
            pred = net_out.data.max(1)[1]  # получаем индекс максимального значения
            correct += pred.eq(target.data).sum()

        # log info   
        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
           test_loss, correct, len(test_loader.dataset),
           100. * correct / len(test_loader.dataset)))

### SGD optimizer

In [292]:
learning_rate = 0.01

# Осуществляем оптимизацию путем стохастического градиентного спуска
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
# Создаем функцию потерь
criterion = nn.NLLLoss()

train(optimizer)  

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.296496
Train Epoch: 0 [2000/60000 (3%)]	Loss: 2.170950
Train Epoch: 0 [4000/60000 (7%)]	Loss: 1.814891
Train Epoch: 0 [6000/60000 (10%)]	Loss: 1.316023
Train Epoch: 0 [8000/60000 (13%)]	Loss: 0.920075
Train Epoch: 0 [10000/60000 (17%)]	Loss: 0.673121
Train Epoch: 0 [12000/60000 (20%)]	Loss: 0.574375
Train Epoch: 0 [14000/60000 (23%)]	Loss: 0.319867
Train Epoch: 0 [16000/60000 (27%)]	Loss: 0.430699
Train Epoch: 0 [18000/60000 (30%)]	Loss: 0.424188
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.321510
Train Epoch: 0 [22000/60000 (37%)]	Loss: 0.298412
Train Epoch: 0 [24000/60000 (40%)]	Loss: 0.285107
Train Epoch: 0 [26000/60000 (43%)]	Loss: 0.352357
Train Epoch: 0 [28000/60000 (47%)]	Loss: 0.370479
Train Epoch: 0 [30000/60000 (50%)]	Loss: 0.296379
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.276954
Train Epoch: 0 [34000/60000 (57%)]	Loss: 0.357670
Train Epoch: 0 [36000/60000 (60%)]	Loss: 0.292101
Train Epoch: 0 [38000/60000 (63%)]	Loss: 0.238567
Train Epoch

Train Epoch: 5 [30000/60000 (50%)]	Loss: 0.097833
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.072328
Train Epoch: 5 [34000/60000 (57%)]	Loss: 0.103291
Train Epoch: 5 [36000/60000 (60%)]	Loss: 0.108415
Train Epoch: 5 [38000/60000 (63%)]	Loss: 0.105498
Train Epoch: 5 [40000/60000 (67%)]	Loss: 0.075708
Train Epoch: 5 [42000/60000 (70%)]	Loss: 0.055130
Train Epoch: 5 [44000/60000 (73%)]	Loss: 0.024565
Train Epoch: 5 [46000/60000 (77%)]	Loss: 0.057736
Train Epoch: 5 [48000/60000 (80%)]	Loss: 0.092228
Train Epoch: 5 [50000/60000 (83%)]	Loss: 0.037179
Train Epoch: 5 [52000/60000 (87%)]	Loss: 0.093136
Train Epoch: 5 [54000/60000 (90%)]	Loss: 0.059731
Train Epoch: 5 [56000/60000 (93%)]	Loss: 0.058384
Train Epoch: 5 [58000/60000 (97%)]	Loss: 0.073063
Train Epoch: 6 [0/60000 (0%)]	Loss: 0.097358
Train Epoch: 6 [2000/60000 (3%)]	Loss: 0.048211
Train Epoch: 6 [4000/60000 (7%)]	Loss: 0.032229
Train Epoch: 6 [6000/60000 (10%)]	Loss: 0.022491
Train Epoch: 6 [8000/60000 (13%)]	Loss: 0.061589
Train Epoch

In [293]:
if __name__ == '__main__':
    main()


Test set: Average loss: 0.0003, Accuracy: 9777/10000 (97%)



### Adam optimizer

In [294]:
learning_rate = 0.01

optimizer = optim.Adam(net.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

train(optimizer)

Train Epoch: 0 [0/60000 (0%)]	Loss: 0.048584
Train Epoch: 0 [2000/60000 (3%)]	Loss: 0.993558
Train Epoch: 0 [4000/60000 (7%)]	Loss: 0.520789
Train Epoch: 0 [6000/60000 (10%)]	Loss: 0.450685
Train Epoch: 0 [8000/60000 (13%)]	Loss: 0.340326
Train Epoch: 0 [10000/60000 (17%)]	Loss: 0.260818
Train Epoch: 0 [12000/60000 (20%)]	Loss: 0.275898
Train Epoch: 0 [14000/60000 (23%)]	Loss: 0.215999
Train Epoch: 0 [16000/60000 (27%)]	Loss: 0.191310
Train Epoch: 0 [18000/60000 (30%)]	Loss: 0.186420
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.245189
Train Epoch: 0 [22000/60000 (37%)]	Loss: 0.282442
Train Epoch: 0 [24000/60000 (40%)]	Loss: 0.251075
Train Epoch: 0 [26000/60000 (43%)]	Loss: 0.185695
Train Epoch: 0 [28000/60000 (47%)]	Loss: 0.168624
Train Epoch: 0 [30000/60000 (50%)]	Loss: 0.190308
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.201288
Train Epoch: 0 [34000/60000 (57%)]	Loss: 0.296966
Train Epoch: 0 [36000/60000 (60%)]	Loss: 0.287170
Train Epoch: 0 [38000/60000 (63%)]	Loss: 0.143276
Train Epoch

Train Epoch: 5 [30000/60000 (50%)]	Loss: 0.102149
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.126936
Train Epoch: 5 [34000/60000 (57%)]	Loss: 0.199733
Train Epoch: 5 [36000/60000 (60%)]	Loss: 0.081604
Train Epoch: 5 [38000/60000 (63%)]	Loss: 0.102584
Train Epoch: 5 [40000/60000 (67%)]	Loss: 0.121032
Train Epoch: 5 [42000/60000 (70%)]	Loss: 0.113066
Train Epoch: 5 [44000/60000 (73%)]	Loss: 0.086816
Train Epoch: 5 [46000/60000 (77%)]	Loss: 0.038283
Train Epoch: 5 [48000/60000 (80%)]	Loss: 0.072193
Train Epoch: 5 [50000/60000 (83%)]	Loss: 0.044043
Train Epoch: 5 [52000/60000 (87%)]	Loss: 0.140496
Train Epoch: 5 [54000/60000 (90%)]	Loss: 0.154627
Train Epoch: 5 [56000/60000 (93%)]	Loss: 0.135093
Train Epoch: 5 [58000/60000 (97%)]	Loss: 0.061793
Train Epoch: 6 [0/60000 (0%)]	Loss: 0.102533
Train Epoch: 6 [2000/60000 (3%)]	Loss: 0.040303
Train Epoch: 6 [4000/60000 (7%)]	Loss: 0.055732
Train Epoch: 6 [6000/60000 (10%)]	Loss: 0.083401
Train Epoch: 6 [8000/60000 (13%)]	Loss: 0.041042
Train Epoch

In [295]:
if __name__ == '__main__':
    main()


Test set: Average loss: 0.0008, Accuracy: 9636/10000 (96%)

